# Imports

In [1]:
import os
from pathlib import Path
import pandas as pd
import sys
sys.path.append('..')
from vars import *
from fuzzywuzzy import process, fuzz
import re

# Read Mapping System

In [2]:
ms_df = pd.read_excel(MAPPING_SYSTEM_PATH, sheet_name='Vehicle Part Mapping System', header=1)
ms_df.head()

,Market,Make,Model,Range,Derivative,Model Year,Engine Size,Fuel Type,Drive Type,Gear Type,...,Year Release,Discontinued Date,Discontinued Year,Discontinued Mth Number,Discontinued Date Blank,Month_Names.Month name,VHID,Lightstone ID,Phase,Unique ID
0,LCV,AAD,VH400,NaN,ET 2500D P-VAN DSL,1995,2.5,DIESEL,4X2,MAN,...,1995,1995-12,1995,12,1995-12,Dec,VHID000001,LSD000001,PHASE 3,VHID000001-1995
1,BUS,AAD,BUS,NaN,17M250A 447N M-ENG 4X2 BUS,1997,0.0,DIESEL,4X2,MAN,...,1997,1998-12,1998,12,1998-12,Dec,VHID000002,LSD000002,PHASE 3,VHID000002-1997
2,BUS,AAD,BUS,NaN,17M250A 447N M-ENG 4X2 BUS,1998,0.0,DIESEL,4X2,MAN,...,1997,1998-12,1998,12,1998-12,Dec,VHID000002,LSD000002,PHASE 3,VHID000002-1998
3,XHV,AAD,DAF,NaN,FTT85-360 6X4 T-T,1996,0.0,DIESEL,6X4,MAN,...,1996,1997-12,1997,12,1997-12,Dec,VHID000003,LSD000003,PHASE 3,VHID000003-1996
4,XHV,AAD,DAF,NaN,FTT85-360 6X4 T-T,1997,0.0,DIESEL,6X4,MAN,...,1996,1997-12,1997,12,1997-12,Dec,VHID000003,LSD000003,PHASE 3,VHID000003-1997


In [3]:
MAKES = ms_df['Make'].unique()
MODELS = ms_df['Model'].unique()

# Read XLSX data

1.  Read xlsx files
2.  remove re-work dataframes that have more than 12 columns
3.  create one long dataframe
4.  rename columns

In [140]:
parent = '/Users/omrijacobsz/otomatika/voomer/vmr1-monroe-data-mapping/notebooks/processed'
files = os.listdir(parent)
for file in files:
    if not '.xlsx' in file:
        files.remove(file)

files

['Table-Extraction_monroe-split-104-to-113.xlsx',
 'Table-Extraction_monroe-split-34-to-43.xlsx',
 'Table-Extraction_monroe-split-84-to-93.xlsx',
 'Table-Extraction_monroe-split-64-to-73.xlsx',
 'Table-Extraction_monroe-split-74-to-83.xlsx',
 'Table-Extraction_monroe-split-94-to-103.xlsx',
 'Table-Extraction_monroe-split-54-to-63.xlsx',
 'Table-Extraction_monroe-split-4-to-13.xlsx',
 'Table-Extraction_monroe-split-44-to-53.xlsx',
 'Table-Extraction_monroe-split-14-to-24.xlsx',
 'Table-Extraction_monroe-split-14-to-23.xlsx',
 'Table-Extraction_monroe-split-4-to-14.xlsx',
 'Table-Extraction_monroe-split-24-to-33.xlsx']

In [141]:
dfs = []
for file in files:
    xls = pd.ExcelFile(os.path.join(parent, file), engine='openpyxl')
    for sheet in range(10):
        dfs.append(pd.read_excel(
            xls,
            'Sheet{}'.format(sheet+1),
            skiprows=3,
            skipfooter=4,
        ))

In [142]:
dfs[0].head()

,Year,Application,Entry\nshocks,Premium\nshocks,Springs,Protection\nkits/\nMounting\nkits,Mounting information,Entry\nshocks.1,Premium\nshocks.1,Springs.1,Protection\nkits/\nMounting\nkits.1,Mounting information.1
0,TOYOTA continued,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lite Ace / Town Ace / Noah Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,96-07,Lite Ace / Town Ace / Noah R50 AWD\nQ,NaN,NaN,NaN,NaN,NaN,“D4468”,NaN,NaN,NaN,U L
3,MR2 C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-07,MR2 cabriolet,—,—,—,—,NaN,—,“72502ST” R\n “72503ST” L,—,—,J


In [143]:
fdfs = []
for df in dfs:
    if df.shape[1] > 12:
        df['Mounting information.1'] = df['Mounting information.1'].combine_first(df['Unnamed: 12'])
        df = df.drop(columns='Unnamed: 12')
        # display(df)
    fdfs.append(df)

In [144]:
df = pd.concat(fdfs)

In [145]:
df

,Year,Application,Entry\nshocks,Premium\nshocks,Springs,Protection\nkits/\nMounting\nkits,Mounting information,Entry\nshocks.1,Premium\nshocks.1,Springs.1,Protection\nkits/\nMounting\nkits.1,Mounting information.1
0,TOYOTA continued,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lite Ace / Town Ace / Noah Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,96-07,Lite Ace / Town Ace / Noah R50 AWD\nQ,NaN,NaN,NaN,NaN,NaN,“D4468”,NaN,NaN,NaN,U L
3,MR2 C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-07,MR2 cabriolet,—,—,—,—,NaN,—,“72502ST” R\n “72503ST” L,—,—,J
...,...,...,...,...,...,...,...,...,...,...,...,...
30,FAW (FIRST AUTO WORKS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Pick up P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,08-16,FAW small pick up,—,—,—,—,—,—,—,—,NaN,NaN
33,Sirius S80 F V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
column_headings = [
    'Year',
    'Application',
    'Front Entry Shock',
    'Front Premium Shock',
    'Front Springs',
    'Front Protection kits/ Mounting kits',
    'Front Mounting information',
    'Rear Entry Shock',
    'Rear Premium Shock',
    'Rear Springs',
    'Rear Protection kits/ Mounting kits',
    'Rear Mounting information',
]

In [147]:
df = df.rename(columns={
    'Year': 'Year',
    'Application': 'Application',
    'Entry\nshocks': 'Front Entry Shock',
    'Premium\nshocks': 'Front Premium Shock',
    'Springs': 'Front Springs',
    'Protection\nkits/\nMounting\nkits': 'Front Protection kits/ Mounting kits',
    'Mounting information': 'Front Mounting information',
    'Entry\nshocks.1': 'Rear Entry Shock',
    'Premium\nshocks.1': 'Rear Premium Shock',
    'Springs.1': 'Rear Springs',
    'Protection\nkits/\nMounting\nkits.1': 'Rear Protection kits/ Mounting kits',
    'Mounting information.1': 'Rear Mounting information',
})[column_headings]

# Clean dataframe

In [148]:
df.Year.fillna(method='ffill', inplace=True)

In [149]:
if []:
    display(True)

In [150]:
no_match_found = []
make_matches = {}
model_matches = {}
for i, row in df.iterrows():
    if '-' not in row['Year']:
        row_make_or_model = row['Year'].rstrip(' continued')
        row_match = process.extractOne(row_make_or_model, MAKES, score_cutoff=85)
        if row_match:
            make_matches[row_match[0]] = row_make_or_model
        else:
            row_match = process.extractOne(row_make_or_model, MODELS, score_cutoff=85)
            if row_match:
                model_matches[row_match[0]] = row_make_or_model
            else:
                no_match_found.append(row_make_or_model)
    

In [151]:
len(no_match_found)

39

In [152]:
process.extract('RAV4', ['RAV'])

[('RAV', 86)]

In [153]:
no_match_found

['Stout P',
 'How to measure & identify the front OE Struts &\nrear OE shocks for Golf 7',
 'How to measure & identify the front OE Struts &\nrear OE shocks for Golf 7',
 'Cortina P',
 'Sierra / Sapphir',
 'Conqueror P',
 'Lion P',
 'GEELY',
 'Zaﬁra Q',
 'Hor',
 'Sep',
 'Sep',
 'SEC C126 & C140 K',
 '* Measurement taken from centre of bottom camber bolt hole to top of strut body',
 'ZR H',
 '01\nmidyear\n’02',
 '280ZX / 300ZX K',
 'Calibra K',
 'FOR JEEP CJ – See Willys at the end of the alphabetical section after Volv',
 'JMC (JIANGLING MOTORS CORPORATION)',
 'Carnival Q',
 'Thema',
 'Other Lancia models, ie Dedra, Delta, Kappa, Lybra, Thema etc., available on special order',
 'Replacement kit when exchanging the air spring\non the shockabsorber (part number is AK247)',
 'Replacement kit when exchanging the air spring\non the shockabsorber (part number is AK247)',
 'LIFAN (Sold via CMC Calibra Motor Corp)',
 'Plutus P',
 'Stelv',
 'Odyssey / Shuttle Q',
 'Stelv',
 'Costantia / Komma',

In [154]:
len(make_matches)

65

In [155]:
len(model_matches)

438

In [156]:
def get_make_or_model(row, makes_or_models, model=True, check_col=False):
    if '-' not in row['Year']:
        matches = []
        row_make_or_model = row['Year'].rstrip(' continued')
        if model:
            row_make_or_model = row_make_or_model.rstrip(' K').rstrip(' H').rstrip(' S').rstrip(' W').rstrip(' F')
        # for make_or_model in makes_or_models:
        #     if make_or_model in row_make_or_model.upper():
        #         matches.append(make_or_model)
        #     else:
        #         print('Row make or model not found:', row_make_or_model)
        #         print(row_make_or_model, process.extractBests(row_make_or_model, makes_or_models, score_cutoff=70))

        row_match = process.extractOne(row_make_or_model, makes_or_models, score_cutoff=85)
        if row_match:
            best_match = process.extractOne(row_make_or_model, makes_or_models)
            if check_col and best_match[1] < 90:
                # print('Low match:', row['Year'])
                return 'Check'
            elif check_col:
                return 'In DB'
            return row_match[0]
        # else:
        #     print('No match:', row_make_or_model)

        # else:
            # print('No matches:', row['Year'])
    
    return None

In [157]:
new_df = df.copy()

In [158]:
df['Make'] = df.apply(lambda row: get_make_or_model(row, MAKES, model=False), axis=1)
df['Model'] = df.apply(lambda row: get_make_or_model(row, MODELS, model=True), axis=1)
df['Check Model'] = df.apply(lambda row: get_make_or_model(row, MODELS, model=True, check_col=True), axis=1)

In [159]:
df.Model.fillna(method='ffill', inplace=True)
df.Make.fillna(method='ffill', inplace=True)
df['Check Model'].fillna(method='ffill', inplace=True)

In [160]:
df.shape

(3506, 15)

In [161]:
df.dropna(
    subset=[
        'Front Entry Shock',
        'Front Premium Shock',
        'Front Springs',
        'Front Protection kits/ Mounting kits',
        'Front Mounting information',
        'Rear Entry Shock',
        'Rear Premium Shock',
        'Rear Springs',
        'Rear Protection kits/ Mounting kits',
        'Rear Mounting information'
    ],
    how='all',
    inplace=True
)

In [162]:
df.shape

(2100, 15)

In [163]:
df = df.assign(Year=df['Year'].str.split('/')).explode('Year')

In [164]:
def get_start_or_end_year(row, start_year=True):
    """
    Returns either the starting year or ending. If `start_year` is set to True then the starting
    year is returned, otherwise the ending year is returned.
    """
    if start_year:
        i = 0
    else:
        i = 1
    
    _year = str(row['Year'])
    _year = _year.replace('only', '')

    if '-' in _year:
        try:
            _year = _year.split('-')[i]

        except IndexError:
            if not start_year:
                return row['Start Year']
            else:
                return None

    if 'All' in _year:
        if start_year:
            _year = 1900
        else:
            _year = 'on'

    elif 'on' not in _year:
        _year = re.sub('\D', '', _year)
        try:
            _year = int(_year)
            if _year < 30:
                _year += 2000
            else:
                _year += 1900
        except ValueError:
            print('Value error with year:', row['Year'])

    if _year == '':
        print('Year:', row['Year'])

    if isinstance(_year, int):
        if _year > 2021:
            _year = ''

    elif isinstance(_year, str):
        if _year != 'on':
            _year = ''
        
    return _year

In [165]:
df['Start Year'] = df.apply(lambda row: get_start_or_end_year(row), axis=1)
df['End Year'] = df.apply(lambda row: get_start_or_end_year(row, start_year=False), axis=1)

Value error with year: Kombi 
Year: Kombi 
Value error with year:  Microbus 
Year:  Microbus 
Value error with year:  Transporter 
Year:  Transporter 
Value error with year:  Transporter pick-up (All rear aircooled engines) M
Year:  Transporter pick-up (All rear aircooled engines) M
Value error with year: (Gti from
Year: (Gti from
Value error with year: Touran
Year: Touran
Value error with year: Croma
Year: Croma
Value error with year: Partner Q
Year: Partner Q
Value error with year: Clio III H
Year: Clio III H
Value error with year: Safrane S
Year: Safrane S
Value error with year: Micra 
Year: Micra 
Value error with year:  March H
Year:  March H
Value error with year: Movano X
Year: Movano X
Value error with year: Hilux Pick-up continued
Year: Hilux Pick-up continued
Value error with year: Hilux Pick-up continued
Year: Hilux Pick-up continued
Value error with year: Scorpio F
Year: Scorpio F
Value error with year:  G SERIES K C
Year:  G SERIES K C
Value error with year: Xsara H
Year: 

In [166]:
df.head()

,Year,Application,Front Entry Shock,Front Premium Shock,Front Springs,Front Protection kits/ Mounting kits,Front Mounting information,Rear Entry Shock,Rear Premium Shock,Rear Springs,Rear Protection kits/ Mounting kits,Rear Mounting information,Make,Model,Check Model,Start Year,End Year
2,96-07,Lite Ace / Town Ace / Noah R50 AWD\nQ,NaN,NaN,NaN,NaN,NaN,“D4468”,NaN,NaN,NaN,U L,TOYOTA,SUPER ACE,Check,1996,2007
4,01-07,MR2 cabriolet,—,—,—,—,NaN,—,“72502ST” R\n “72503ST” L,—,—,J,TOYOTA,C,In DB,2001,2007
6,90-’96,“Grey” Import Old Prado J70 series S/W\n(RJ / ...,—,“D6434”\n(‘90-May’93)\nor “D6438”\n(May’93-’96),—,—,M L\nSpecify year,—,“D6435”\n(‘90-May’93)\nor “D6439”\n(May’93-’96),—,—,M L\nSpecify year,TOYOTA,SUPRA,In DB,1990,1996
7,97-’02,Prado J90 series 3 door SWB S/W (EXCL\neye/eye...,M8077,RS999762,SP1943 (std ht),—,U M\nS/Pan unit (reuse OE\nwashers),—,D8343 /\nRS999187,SP1948 (std ht),—,U R,TOYOTA,SUPRA,In DB,1997,2002
8,97-’02,Prado J95 series 5 door LWB S/W (EXCL\neye/eye...,M8078,RS999762,SP1945 (std ht),—,U M\nS/Pan unit (reuse OE\nwashers),—,D8343 /\nRS999187,SP1950 (std ht),—,U R,TOYOTA,SUPRA,In DB,1997,2002


In [167]:
def extract_incl_derivatives(row):
    deriv = str(row.Application).replace(str(row.Model).replace('/', '&'), '').strip()
    init_deriv = deriv.replace('\r', '')
    deriv = init_deriv
    if '(EXCL' in deriv.upper():
        deriv_i = deriv.upper().find('(EXCL')
        deriv = deriv[:deriv_i].strip()
        
    # print('Deriv before', deriv)
    if '(INCL' in init_deriv.upper():
        # print('Incl found in', init_deriv)
        other_includes_i = init_deriv.upper().find('(INCL')
        deriv = deriv[:other_includes_i]
        deriv = [x.strip() for x in re.split(',|&', deriv)]
        # print('Deriv after', deriv)
        other_includes = init_deriv[other_includes_i:].strip()
        # other_includes = other_includes[other_includes_i:].strip()
        other_includes = re.sub('\(incl|\(INCL\.*|\)', '', other_includes)
        other_includes = [x.strip() for x in re.split(',|&|\/', other_includes)]
        # print('Other includes:', other_includes)
        deriv.extend(other_includes)
    else:
        deriv = [x.strip() for x in re.split(',|&|\/', deriv)]

    if isinstance(deriv, list):
        if len(list(filter(None, deriv))) == 0:
            return ''
        else:
            return deriv
    else:
        return ''

In [168]:
def extract_excl_derivatives(row):
    deriv = str(row.Application).replace(str(row.Model).replace('/', '&'), '').strip()
    init_deriv = deriv.replace('\r', '')
    deriv = init_deriv
    if '(EXCL' in deriv.upper():
        deriv_i = deriv.upper().find('(EXCL')
        deriv = deriv[deriv_i:].strip()
        deriv = re.sub('\(excl|\(EXCL\.*|\)', '', deriv)
        deriv = [x.strip() for x in re.split(',|&|\/', deriv)]
    else:
        return ''

    return deriv

In [169]:
df['Include Derivatives'] = df.apply(lambda row: extract_incl_derivatives(row), axis=1)
df['Exclude Derivatives'] = df.apply(lambda row: extract_excl_derivatives(row), axis=1)

# Clean up database

## Remove funny characters from parts

In [170]:
def remove_funnies(row, col):
    value = row[col]
    if isinstance(value, str):
        value = re.sub('\r', ' ', value)
        value = re.sub(r'\“*|\”*', '', value)
        return value
    else:
        return row[col]

In [171]:
part_cols = [
    'Front Entry Shock',
    'Front Premium Shock',
    'Front Springs',
    'Front Protection kits/ Mounting kits',
    'Front Mounting information',
    'Rear Entry Shock',
    'Rear Premium Shock',
    'Rear Springs',
    'Rear Protection kits/ Mounting kits',
    'Rear Mounting information'
]

for col in part_cols:
    df[col] = df.apply(lambda row: remove_funnies(row, col), axis=1)

In [172]:
def extract_with_regex(row, col, search_pattern, sub_pattern):
    value = row[col]
    if value and isinstance(value, str):
        if re.findall(search_pattern, value):
            # value = re.sub(r'\“*|\”*', '', value)
            return re.sub(sub_pattern, '', value)

    return ''

In [173]:
df['Rear Protection kits'] = df.apply(lambda row: extract_with_regex(row, 'Rear Protection kits/ Mounting kits', r'PK\d*', r'\/\s*MK\d*\s*'), axis=1)
df['Rear Mounting kits'] = df.apply(lambda row: extract_with_regex(row, 'Rear Protection kits/ Mounting kits', r'MK\d*', r'\“*PK\d*\”*\s*\/*\s*'), axis=1)
df['Front Protection kits'] = df.apply(lambda row: extract_with_regex(row, 'Front Protection kits/ Mounting kits', r'PK\d*', r'\/\s*MK\d*\s*'), axis=1)
df['Front Mounting kits'] = df.apply(lambda row: extract_with_regex(row, 'Front Protection kits/ Mounting kits', r'MK\d*', r'\“*PK\d*\”*\s*\/*\s*'), axis=1)

In [174]:
fdf = df[[
    'Make',
    'Model',
    'Check Model',
    'Start Year',
    'End Year',
    'Include Derivatives',
    'Exclude Derivatives',
    'Front Entry Shock',
    'Front Premium Shock',
    'Front Springs',
    'Front Protection kits',
    'Front Mounting kits',
    'Front Mounting information',
    'Rear Entry Shock',
    'Rear Premium Shock',
    'Rear Springs',
    'Rear Protection kits',
    'Rear Mounting kits',
    'Rear Mounting information',
]]

In [175]:
fdf = fdf.assign(Model=fdf['Model'].str.split('/')).explode('Model')

# Match with mapping system

In [176]:
def match_cols(x, row):

    if row['End Year'] == 'on':
        end_year = datetime.datetime.now().year
    else:
        end_year = row['End Year']

    try:
        if isinstance(end_year, str):
            end_year = int(end_year)
    except ValueError:
        print(row)
        return False

    fill = False

    if x.Make == row.Make and\
        x.Model == row.Model and\
            x['Model Year'] > row['Start Year'] and\
                x['Model Year'] < end_year:
                fill = True
                incl_derivs = row['Include Derivatives']
                excl_derivs = row['Exclude Derivatives']
                if incl_derivs:
                    # print('Include Derivs:', incl_derivs)
                    # print( x['Derivative'])
                    if not any(fuzz.WRatio(deriv.upper(), x['Derivative']) >= 90 for deriv in incl_derivs):
                        fill = False
                if excl_derivs:
                    # print('Exclude Derivs', excl_derivs)
                    # print( x['Derivative'])
                    if any(fuzz.WRatio(deriv.upper(), x['Derivative']) >= 90 for deriv in excl_derivs):
                        fill = False
    
    if fill:
        return True
    
    return False

In [177]:
def derivative_match(x, row, include=True):
    if include:
        in_or_ex = 'Include'
    else:
        in_or_ex = 'Exclude'
    incl_derivs = row[f'{in_or_ex} Derivatives']
    for deriv in incl_derivs:
        # print(x.keys())
        if fuzz.WRatio(deriv, x['Derivative']) >= 90:
            return deriv
    return ''

In [178]:
column_mapping = {
    'Front Left': 'Front Entry Shock',
    'Front Right': 'Front Entry Shock',
    'Protection Kits Front': 'Front Protection kits',
    'Mounting Kits Front': 'Front Mounting kits',
    'Rear Left': 'Rear Entry Shock',
    'Rear Right ': 'Rear Entry Shock',
    'Protection Kits Rear': 'Rear Protection kits',
    'Mounting Kits Rear': 'Rear Mounting kits',
    'Front Left 1': 'Front Premium Shock',
    'Front Right 1': 'Front Premium Shock',
    'Rear Left    1': 'Rear Premium Shock',
    'Rear Right 1': 'Rear Premium Shock',
}

In [ ]:
for index, row in fdf.iterrows():

    indexes = ms_df[ms_df.apply(lambda x: match_cols(x, row), axis=1)].index
    
    ms_df.loc[indexes, 'Include Derivative'] = ms_df.loc[indexes].apply(lambda x: derivative_match(x, row, include=True), axis=1)
    ms_df.loc[indexes, 'Exclude Derivative'] = ms_df.loc[indexes].apply(lambda x: derivative_match(x, row, include=False), axis=1)

    for key in column_mapping.keys():
        ms_df.loc[indexes, key] = row[column_mapping[key]]